# Obtain data of AGN from SIMBAD, filtering the objects with data of Ne II, Ne III and Ne V in infrared from NED Phot Tables and saving it

In this notebook we develop the code to Obtain the data of AGNs type Sy1 (Seyfert 1), Sy2 (Seyfert 2) and SyG (Seyfert Galaxies) from **[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-tap)** (SMB) and then search each object from the SMB sample in **[NED](https://ned.ipac.caltech.edu/)** to obtain the photometric data and filter all the objects from the sample that contain data of the emission lines of Ne II, Ne III and Ne V in infrared.

The part of the code to obtain the data from SIMBAD is based in Andres Ramos's work to obtain Seyfert Samples (https://github.com/aframosp/AGNView).

In [1]:
# If this notebook will be run in Google Colab, first at all the next code must be run. If the notebook will be run in Jupyter
#then "comment" or omit this part 

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install astroquery

In [2]:
from datetime import date
import time
import requests
import numpy as np

import pyvo
from astroquery.ipac.ned import Ned
from astroquery.simbad import Simbad

from pathlib import Path
from requests import Request, Session

import astropy.units as u
from astropy.table import hstack, QTable, Table
from astropy.coordinates import SkyCoord

In the next cell we define the paths and create the folders where we will save the data downloaded from the different Data Bases:

In [3]:
path_raw='../Datos/Raw/'
Path(path_raw).mkdir(parents=True, exist_ok=True)
path_phot_ned = '../Datos/Work/Phot_Tables_NED/' #In this path wi will save the photometry tables for each object
Path(path_phot_ned).mkdir(parents=True, exist_ok=True)

Using the package **[PyVO](https://pyvo.readthedocs.io/en/latest/#)** we can access to remote data and services like the **TAP Service of SIMBAD [SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-tap)** to find and retrieve the astronomical data that we need, in this case: Seyfert Galaxies. To achive that, we use a script with **[ADQL](https://www.ivoa.net/documents/latest/ADQL.html) (Astronomical Data Query Language)**, which is based on SQL.

On the script we used the next **Field Names** for VOTable ouput:
- ra: Right ascension
- dec: Declination
- coo_bibcode: Bibliographical reference for coordinates
- otype_txt: Object type
- rvz_bibcode: Bibliographical reference of radial velocity and redshift
- rvz_radvel: Stored value. Either a radial velocity, or a redshift, depending on the rvz_type field 
- rvz_redshift
- rvz_type: Stored type of velocity: 'v'=radial velocity, 'z'=redshift
- nbref: bibliographical reference

~~~~sql
SELECT main_id
	,ra
	,dec
	,coo_bibcode
	,otype_txt
	,rvz_bibcode
	,rvz_radvel
	,rvz_redshift
	,rvz_type
	,nbref
	,alltypes.otypes
FROM basic
JOIN alltypes ON oid = oidref
WHERE basic.otype = 'Sy1'
	OR basic.otype = 'Sy2'
	OR basic.otype = 'SyG'
~~~~

In [4]:
service = pyvo.dal.TAPService("http://simbad.u-strasbg.fr:80/simbad/sim-tap")

result = service.search("""
SELECT main_id
    ,ra
    ,dec
    ,coo_bibcode
    ,otype_txt
    ,rvz_bibcode
    ,rvz_radvel
    ,rvz_redshift
    ,rvz_type
    ,nbref
    ,alltypes.otypes
FROM basic
JOIN alltypes ON oid = oidref
WHERE basic.otype = 'Sy1'
    OR basic.otype = 'Sy2'
    OR basic.otype = 'SyG'
""")

We extract the object name (or in this case the main id) from the sample result.

In [5]:
simbad_sample_Sy = result.to_table()
obj_name_SMB=simbad_sample_Sy["main_id"]
len(simbad_sample_Sy)

44272

We save the date in a variable for use later in the file names of the results

In [6]:
today = date.today()
today.strftime('%b_%d_%Y')

'Apr_12_2022'

In [7]:
INIT_URL = 'http://vizier.u-strasbg.fr/viz-bin/sed?-c='
urls = [INIT_URL+str(simbad_sample_Sy['ra'][row])+'%20'+
        str(simbad_sample_Sy['dec'][row])+'&-c.rs=5' for row in range(len(simbad_sample_Sy))]
simbad_sample_Sy['cds_url'] = urls

In [8]:
simbad_sample_Sy

main_id,ra,dec,coo_bibcode,otype_txt,rvz_bibcode,rvz_radvel,rvz_redshift,rvz_type,nbref,otypes,cds_url
,deg,deg,,,,km / s,,,,,
object,float64,float64,object,object,object,float64,float64,str1,int32,object,str93
RX J1203.5+4432,180.88724599190996,2.4929432724600002,2020yCat.1350....0G,Sy1,2012ApJS..203...21A,22264.155457144552,0.07724,z,42,* |AGN|EmO|G |GiC|IR |QSO|Rad|Sy1|V* |X,http://vizier.u-strasbg.fr/viz-bin/sed?-c=180.88724599190996%202.4929432724600002&-c.rs=5
2MASX J12014361+4307386,180.43169828928998,43.1273394439,2020yCat.1350....0G,Sy2,2009ApJS..182..543A,20748.633978793205,0.07178,z,4,* |G |Rad|Sy2|rG,http://vizier.u-strasbg.fr/viz-bin/sed?-c=180.43169828928998%2043.1273394439&-c.rs=5
SDSS J144452.42+511957.5,221.218431,51.332651,2012ApJS..203...21A,Sy2,2015ApJS..219...12A,25457.504881087,0.08885,z,4,G |GiG|Sy2,http://vizier.u-strasbg.fr/viz-bin/sed?-c=221.218431%2051.332651&-c.rs=5
FIRST J083344.2+283939,128.43421094374,28.66111602879,2020yCat.1350....0G,Sy1,2009ApJS..182..543A,173513.43668282355,0.936,z,13,* |AGN|QSO|Rad|Sy1,http://vizier.u-strasbg.fr/viz-bin/sed?-c=128.43421094374%2028.66111602879&-c.rs=5
SDSS J091658.99+012604.8,139.24580903083,1.43466726489,2020yCat.1350....0G,Sy1,2015ApJS..219...12A,125231.50632150602,0.56039,z,8,* |QSO|Sy1,http://vizier.u-strasbg.fr/viz-bin/sed?-c=139.24580903083%201.43466726489&-c.rs=5
MCG+06-34-028,235.23225416666668,32.78155555555555,2006AJ....131.1163S,Sy2,2002LEDA.........0P,15618.951821140114,0.05353,z,12,G |IR |Sy2,http://vizier.u-strasbg.fr/viz-bin/sed?-c=235.23225416666668%2032.78155555555555&-c.rs=5
IC 3834,192.88488958362,-14.221116888659996,2020yCat.1350....0G,Sy1,2002LEDA.........0P,4332.318180333962,0.014557,z,34,* |AGN|G |GiG|IR |Sy1,http://vizier.u-strasbg.fr/viz-bin/sed?-c=192.88488958362%20-14.221116888659996&-c.rs=5
SDSS J155011.46+233017.1,237.54777578073,23.50475679549,2020yCat.1350....0G,Sy2,2012ApJS..203...21A,33510.40645237621,0.11879,z,1,* |Sy2,http://vizier.u-strasbg.fr/viz-bin/sed?-c=237.54777578073%2023.50475679549&-c.rs=5


The sample results is saved in a .vot file:

In [9]:
simbad_sample_Sy.write(path_raw+'SMB_'+today.strftime('%b_%d_%Y')+'_Sy_Samples'+'.vot',
                    format='votable',overwrite=True)

### Function to download the Photometry tables from [NED](https://ned.ipac.caltech.edu/).

To get the photometry tables from NED, we use **[Astroquery](https://astroquery.readthedocs.io/en/latest/index.html)** package. In the function **photNED** first we try to obtain the photometry table with the ID name of the object from the SIMBAD sample. If there is not result, then with Astroquery we will search the others objects ID, and with that, search the information in NED. If finallly is not possible to find information with the objects ID, then we will try using its coordinates.

In [10]:
def photNED(name,coord):
    
    '''
    This function is to get the photometry tables
    from NED. The input are the id and coordinates
    of the object.
    '''
    
    phot_ned = QTable()
    id_used = []
    
    try:
        phot_ned = Ned.get_table(name, table='photometry') #We get the Photometry Table using first SIMBAD names
        id_used = name
    except:
        other_ids = Simbad.query_objectids(name) #We get other IDs for the object from SIMBAD
        for other_id in other_ids['ID']:
            try:
                phot_ned = Ned.get_table(other_id, table='photometry') #Photometry Table using other SIMBAD names
                id_used = other_id
                break
            except:
                pass
    if id_used == []:
        try:
            region = Ned.query_region(coord,radius= 5 * u.arcsec)
            id_ned = region["Object Name"]
            phot_ned = Ned.get_table(id_ned, table='photometry')
            id_used = id_ned
        except:
            pass

    return phot_ned, id_used

Using **SkyCoord** from astropy we extract the coordinates information for each object from the SIMBAD samples. This is useful if we want to search information in [NED](https://ned.ipac.caltech.edu/) using coordinates instead the object name

In [11]:
cat_smb_sample = SkyCoord(ra=simbad_sample_Sy['ra'], dec=simbad_sample_Sy['dec'])

### Filtering the Sample: Objects with [Ne II], [Ne III] and [Ne V] information. Downloading the NED Tables:

The wavelenght for NeII, NeIII and NeV in the infrared spectrum are 12.8 microns, 15.6 microns and 14.3 microns respectively. This values are equivalents to the following frequencies which ara related to different labels of "Observed Passband" in NED database as we ilustrate in the next list:

* Frequency: 1.92e+13 Hz: [NeIII] 15.6 microns
* Frecuency: 1.93e+13 Hz: [Ne III] 15.56 (IRS), [NeIII] 15.6 Spitzer, [Ne III] (Spitzer), [Ne III] 15.5 (IRS)
* Frecuency: 2.09e+13 Hz: [Ne V] 14.32 (IRS), [Ne V] 14.32 Spitzer, [Ne V] 14.32, [Ne V] 14.3 Spitzer, [Ne V] 14.3 microns
* Frequency: 2.10e+13 Hz: [Ne V] 14.3 (IRS)
* Frequency: 2.34e+13 Hz: [Ne II] 12.81 (IRS), [Ne II] 12.8 Spitzer, [Ne II] 12.8 (IRS), [Ne II] 12.81, NEII (VLT), 12.81 microns (VLT) 

In the variable **Ne_IR_Fq** we define a list with the those frequency values

In [12]:
Ne_IR_Fq = np.array([19200000000000.0,19300000000000.0,20900000000000.0,21000000000000.0,23400000000000.0])*u.Hz

The filter for the objects with Ne Line Emissions in Infrared will be looking in the column **"Frequency"** of the photometry tables that we got from NED the values of the list defined in Ne_IR_Freq. Then, in a second fliter we are going to classify according to what information the Ne II, Ne III and Ne V emission lines each have.

In [13]:
NeIR_obj_t = QTable(names=('Main_Id','Id_used_NED','RA','DEC','otype_txt','redshift',
                           'rvz_type','otypes','cds_url','SMB_sample_index'),
                    dtype=('O','O','f8','f8','O','f8','U1','O','U93','f8'))

Ne_Inf = []

print(f"{'#':4} {'Idx':6} {'Name ID':^27} {'ID used in NED':^27} {'Ne IR Info':^20}")
i=-1
start = time.time()
for ind, id_smb in enumerate(obj_name_SMB):
    try:       
        #print('Idx SIMBAD: ',ind)
        phot_t,name_ned = photNED(id_smb,cat_smb_sample[ind])
        Fr = np.array([phot_t['Frequency']])*u.Hz #We get the Frequency column from the table
        ints = np.intersect1d(Ne_IR_Fq, Fr) #Intersection between Fr and Ne_IR_Fq
        if len(ints)>0: #First filter: If the phot table has infomation of frequencies of Fr
            i=i+1
            phot_t.write(path_phot_ned+id_smb+'_NED_phot_tables'+'.vot', #Save the phot table in a vot table file
                         format='votable',overwrite=True)
            NeIR_obj_row = [id_smb,name_ned,simbad_sample_Sy["ra"][ind],simbad_sample_Sy["dec"][ind],
                                   simbad_sample_Sy["otype_txt"][ind],simbad_sample_Sy["rvz_redshift"][ind],
                                   simbad_sample_Sy["rvz_type"][ind],simbad_sample_Sy["otypes"][ind],
                                   simbad_sample_Sy["cds_url"][ind],ind] 
            NeIR_obj_t.add_row(vals=NeIR_obj_row)
            
            phot_t.remove_rows(np.where(phot_t['Flux Density'].mask)[0])
            #phot_t.remove_rows(np.where(phot_t['NED Uncertainty'] == '')[0])
            #phot_t.remove_rows(np.where(phot_t['NED Uncertainty'] == '+/-...')[0])
            Fq = np.array([phot_t['Frequency']])*u.Hz
            ins = np.intersect1d(Ne_IR_Fq, Fq) #Intersection between Fq and Ne_IR_Fq
            if (Ne_IR_Fq[0] in ins or Ne_IR_Fq[1] in ins)and (Ne_IR_Fq[2] in ins or Ne_IR_Fq[3] in ins)\
            and Ne_IR_Fq[4] in ins: #Second filter to classify according the Ne info after remove empty flux values
                Ne_info = 'NeII|NeIII|NeV'
            elif (Ne_IR_Fq[0] in ins or Ne_IR_Fq[1] in ins) and Ne_IR_Fq[4] in ins:
                Ne_info = 'NeII|NeIII'
            elif (Ne_IR_Fq[2] in ins or Ne_IR_Fq[3] in ins) and Ne_IR_Fq[4] in ins:
                Ne_info = 'NeII|NeV'
            elif (Ne_IR_Fq[0] in ins or Ne_IR_Fq[1] in ins) and (Ne_IR_Fq[2] in ins or Ne_IR_Fq[3] in ins):
                Ne_info = 'NeIII|NeV'
            elif Ne_IR_Fq[0] in ins or Ne_IR_Fq[1] in ins:
                Ne_info = 'NeIII'
            elif Ne_IR_Fq[2] in ins or Ne_IR_Fq[3] in ins:
                Ne_info = 'NeV'
            elif Ne_IR_Fq[4] in ins:
                Ne_info = 'NeII'
            else:
                Ne_info = 'No Flux Values' 
            Ne_Inf.append(Ne_info)
            
            print(f"{'%g'%i:4} {'%g'%ind:6} {'%s'%id_smb:^27} {'%s'%name_ned:^27} {'%s'%Ne_info:^20}")
    
    except KeyError:
        pass

end = time.time()

print('\nTotal objects with lines emission of NeII or NeIII or NeV in IR  found: ',len(NeIR_obj_t))
print('Execution Time(seg): ' + str(end - start))
print('Execution Time(min): ' + str((end - start)/60))
print('Execution Time(hrs): ' + str((end - start)/3600))

#    Idx              Name ID                 ID used in NED             Ne IR Info     
0    283             3C 234.0                    3C 234.0              NeII|NeIII|NeV   
1    414               M  84                       M  84                    NeV         
2    428      2MASS J13000535+1632148       NVSS J130005+163212          NeII|NeIII     
3    484             NGC   262                   NGC   262             NeII|NeIII|NeV   
4    541          IRAS 04385-0828             IRAS 04385-0828          NeII|NeIII|NeV   
5    628             NGC  4945                   NGC  4945             NeII|NeIII|NeV   
6    654              IC 4553                     IC 4553              NeII|NeIII|NeV   
7    773             NGC  5252                   NGC  5252                  NeII        
8    852             Mrk  279                    Mrk  279              NeII|NeIII|NeV   
9    1227            UGC 11680                   UGC 11680                  NeII        
10   1246          LE

92   11682           Mrk  937                    Mrk  937              No Flux Values   
93   11977   SDSS J040208.86-050642.0    SDSS J040208.86-050642.0      No Flux Values   
94   12122    2MASX J22041914+0333511     2MASX J22041914+0333511      NeII|NeIII|NeV   
95   12331     2XMM J094046.2+033929       2XMM J094046.2+033929       No Flux Values   
96   12371           NGC  7469                   NGC  7469             NeII|NeIII|NeV   
97   12511       IRAS F08216+3009            IRAS F08216+3009          NeII|NeIII|NeV   
98   12549    2MASX J08035923+2345201     2MASX J08035923+2345201          NeIII        
99   12666           NGC  7591                   NGC  7591               NeII|NeIII     
100  12683    2MASS J08105865+7602424     2MASS J08105865+7602424          NeIII        
101  12884         LEDA  166692                LEDA  166692            No Flux Values   
102  12934        IRAS 15091-2107             IRAS 15091-2107          NeII|NeIII|NeV   
103  12980    2MASS J

185  23664             M  49                       M  49                    NeV         
186  23768          ESO 263-13                  ESO 263-13                  NeII        
187  23869    2MASX J21564950-0745325     2MASX J21564950-0745325           NeV         
188  24175         LEDA 2816291                LEDA 2816291                NeIII        
189  24279     ICRF J090933.4+425346       ICRF J090933.4+425346       No Flux Values   
190  24503           NGC  5347                   NGC  5347             NeII|NeIII|NeV   
191  24603           Mrk  520                    Mrk  520                   NeII        
192  24644           NGC  4748                   NGC  4748             NeII|NeIII|NeV   
193  24664           NGC  1667                   NGC  1667             NeII|NeIII|NeV   
194  24886            2C 1883                   1Jy 2314+03              NeII|NeIII     
195  24941           NGC  1320                   NGC  1320             NeII|NeIII|NeV   
196  24989           

279  35799  [VV2003c] J082001.7+505039           SHOC 198                  NeIII        
280  35970    2MASX J08584172+1041223     2MASX J08584172+1041223      NeII|NeIII|NeV   
281  36020   SDSS J231812.99-004126.1    SDSS J231812.99-004126.1          NeIII        
282  36035            3C 343                      3C 343               NeII|NeIII|NeV   
283  36047           NGC  4992                   NGC  4992                  NeII        
284  36085           NGC  5395                   NGC  5395             No Flux Values   
285  36118    2MASS J10272497+1219196     2MASS J10272497+1219196      No Flux Values   
286  36149             M  94                       M  94                 NeII|NeIII     
287  36232           NGC  5990                   NGC  5990             NeII|NeIII|NeV   
288  36317           Mrk    3                    Mrk    3              NeII|NeIII|NeV   
289  36379           NGC  4051                   NGC  4051             NeII|NeIII|NeV   
290  36398         LE

In [21]:
NeIR_obj_t['NED_Ne_IR_info'] = Ne_Inf #Add a column to save the Ne info
NeIR_obj_t.show_in_notebook()

idx,Main_Id,Id_used_NED,RA,DEC,otype_txt,redshift,rvz_type,otypes,cds_url,SMB_sample_index,Ne_IR_info,NED_Ne_IR_info
0,3C 234.0,3C 234.0,150.45634473807,28.7858176354,Sy1,0.18494,z,AGN|G |IR |QSO|Rad|Sy1|X |blu|gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=150.45634473807%2028.7858176354&-c.rs=5,283.0,NeII|NeIII|NeV,NeII|NeIII|NeV
1,M 84,M 84,186.2655972083333,12.886983138888887,Sy2,0.00327,z,* |AGN|G |GiC|GiG|IR |Rad|Sy2|UV |X |smm,http://vizier.u-strasbg.fr/viz-bin/sed?-c=186.2655972083333%2012.886983138888887&-c.rs=5,414.0,NeV,NeV
2,2MASS J13000535+1632148,NVSS J130005+163212,195.02228295163,16.53743305218,Sy1,0.07993,z,AGN|G |IR |Rad|Sy1|X,http://vizier.u-strasbg.fr/viz-bin/sed?-c=195.02228295163%2016.53743305218&-c.rs=5,428.0,NeII|NeIII,NeII|NeIII
3,NGC 262,NGC 262,12.196426138730002,31.956956641230004,Sy2,0.015260233939595302,v,* |AGN|Bla|G |GiG|IR |QSO|Rad|Sy2|X |gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=12.196426138730002%2031.956956641230004&-c.rs=5,484.0,NeII|NeIII|NeV,NeII|NeIII|NeV
4,IRAS 04385-0828,IRAS 04385-0828,70.2290037068,-8.37281297812,Sy2,0.015064,z,* |AGN|G |GiG|IR |Sy2,http://vizier.u-strasbg.fr/viz-bin/sed?-c=70.2290037068%20-8.37281297812&-c.rs=5,541.0,NeII|NeIII|NeV,NeII|NeIII|NeV
5,NGC 4945,NGC 4945,196.3636625,-49.4679,Sy2,0.0017968563040435814,v,AGN|G |GiG|IR |Rad|Sy2|X |cm |gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=196.3636625%20-49.4679&-c.rs=5,628.0,NeII|NeIII|NeV,NeII|NeIII|NeV
6,IC 4553,IC 4553,233.73843316666665,23.50322455555555,SyG,0.0184,z,AGN|G |GiG|GiP|IG |IR |LIN|Rad|SyG|X |gam|smm,http://vizier.u-strasbg.fr/viz-bin/sed?-c=233.73843316666665%2023.50322455555555&-c.rs=5,654.0,NeII|NeIII|NeV,NeII|NeIII|NeV
7,NGC 5252,NGC 5252,204.56612821846997,4.542644796,Sy2,0.02294,z,AG?|AGN|EmG|G |GiG|Sy1|Sy2|X |gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=204.56612821846997%204.542644796&-c.rs=5,773.0,NeII,NeII
8,Mrk 279,Mrk 279,208.26431206837998,69.30816969735,Sy1,0.030601,z,AGN|G |IR |QSO|Sy1|UV |X |gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=208.26431206837998%2069.30816969735&-c.rs=5,852.0,NeII|NeIII|NeV,NeII|NeIII|NeV
9,UGC 11680,UGC 11680,316.93375,3.875277777777778,Sy2,0.026,z,G |GiP|IG |IR |PaG|Sy2,http://vizier.u-strasbg.fr/viz-bin/sed?-c=316.93375%203.875277777777778&-c.rs=5,1227.0,NeII,NeII


In [22]:
NeIR_obj_t.write(path_raw+'Obj_Ne-IR_'+today.strftime('%b_%d_%Y')+'.vot',
                    format='votable',overwrite=True)

In [23]:
NeII_NeIII_NeV_IR = np.where(NeIR_obj_t['Ne_IR_info'] == 'NeII|NeIII|NeV')[0] #List of objects with NeII, NeIII and NeV data
NeII_NeIII_IR = np.where(NeIR_obj_t['Ne_IR_info'] == 'NeII|NeIII')[0] #List of objects with NeII and NeIII data
NeII_NeV_IR = np.where(NeIR_obj_t['Ne_IR_info'] == 'NeII|NeV')[0] #List of objects with NeII and NeV data
NeIII_NeV_IR = np.where(NeIR_obj_t['Ne_IR_info'] == 'NeIII|NeV')[0] #List of objects with NeIII and NeV data
NeIII_IR = np.where(NeIR_obj_t['Ne_IR_info'] == 'NeIII')[0] #List of objects with NeIII data
NeV_IR = np.where(NeIR_obj_t['Ne_IR_info'] == 'NeV')[0] #List of objects with NeV data
NeII_IR = np.where(NeIR_obj_t['Ne_IR_info'] == 'NeII')[0] #List of objects with NeII data
No_Ne_IR = np.where(NeIR_obj_t['Ne_IR_info'] == 'No Flux Values')[0] #List of objects with NeII data
print('Total objects found with NeII, NeIII and NeV emission lines: ',len(NeII_NeIII_NeV_IR))
print('Total objects found with NeII and NeIII emission lines: ',len(NeII_NeIII_IR))
print('Total objects found with NeII and NeV emission lines: ',len(NeII_NeV_IR))
print('Total objects found with NeIII and NeV emission lines: ',len(NeIII_NeV_IR))
print('Total objects found with NeIII emission lines: ',len(NeIII_IR))
print('Total objects found with NeV emission lines: ',len(NeV_IR))
print('Total objects found with NeII emission lines: ',len(NeII_IR))
print('Total objects found with No Ne flux values: ',len(No_Ne_IR))

Total objects found with NeII, NeIII and NeV emission lines:  177
Total objects found with NeII and NeIII emission lines:  57
Total objects found with NeII and NeV emission lines:  13
Total objects found with NeIII and NeV emission lines:  8
Total objects found with NeIII emission lines:  20
Total objects found with NeV emission lines:  20
Total objects found with NeII emission lines:  41
Total objects found with No Ne flux values:  23


## Notebook Info

In [17]:
%reload_ext watermark
%watermark -a "Jonhatan Bernal" -d -v -m
print('Specific Python packages')
%watermark -iv -w --packages astropy

Author: Jonhatan Bernal

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 8.2.0

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.13.0-39-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Specific Python packages
astropy: 5.0

astropy : 5.0
pyvo    : 1.2.1
numpy   : 1.20.3
requests: 2.27.1
sys     : 3.9.7 (default, Sep 16 2021, 13:09:58) 
[GCC 7.5.0]

Watermark: 2.3.0



### Las celdas a continuación son para pruebas:

In [18]:
x=428
simbad_sample_Sy[x]

main_id,ra,dec,coo_bibcode,otype_txt,rvz_bibcode,rvz_radvel,rvz_redshift,rvz_type,nbref,otypes,cds_url
,deg,deg,,,,km / s,,,,,
object,float64,float64,object,object,object,float64,float64,str1,int32,object,str93
2MASS J13000535+1632148,195.02228295163,16.53743305218,2020yCat.1350....0G,Sy1,2012ApJS..203...21A,23007.57777582905,0.07993,z,45,AGN|G |IR |Rad|Sy1|X,http://vizier.u-strasbg.fr/viz-bin/sed?-c=195.02228295163%2016.53743305218&-c.rs=5


In [19]:
nam=simbad_sample_Sy['main_id'][x]
other_ids = Simbad.query_objectids(nam)
other_ids

ID
bytes29
Gaia EDR3 3937375195178654720
TIC 88137340
2MASS J13000535+1632148
NVSS J130005+163212
2MASX J13000533+1632151
2MASSI J1300053+163214
2MASS J13000534+1632149
FIRST J130005.3+163214
[VV2003c] J130005.3+163214


In [20]:
for _id in other_ids['ID']:
    try:
        ned_tab= Ned.get_table(_id, table='photometry')
        print('si ',_id)
        break   
    except:
        print('no ',_id)
        if _id == other_ids['ID'][-1]:
            continue

no  Gaia EDR3 3937375195178654720
no  TIC 88137340
no  2MASS J13000535+1632148
si  NVSS J130005+163212
